In [2]:
import os
import sys
from sys import platform
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns

# Add the parent directory to the sys.path (without this we cannot import constants or scripts)
sys.path.insert(0, str(Path.cwd().parent))
from src.plotting import plot_comparison_GER_HIC_GLO
from constants.countries import highincome_countries
from src.utils import get_iso3_gbd

%load_ext autoreload

In [3]:
import matplotlib.pyplot as plt 

from tueplots import bundles
plt.rcParams.update(bundles.icml2022(column="full", ncols=1, nrows=1))

In [4]:
# if platform == 'darwin': # macos
#     print('mac os')
#     os.environ["PATH"] += os.pathsep + '/Library/TeX/texbin'

In [5]:
DATA_PATH = '../data/final'
OUTPUT_PATH = '../doc/IHD_germany_2024/fig'
cvd_data_path = f'{DATA_PATH}/gbd_cardiovascular_allAges_final.csv'
YEAR_COLUMN_NAME = 'year'
VALUE_COLUMN_NAME = 'val'

official_range = range(1990, 2020)

## Load Data

In [6]:
df = pd.read_csv(cvd_data_path)
df = df[(df.measure_name=='Incidence') & (df['metric_name'] == 'Rate')]
df['country_code'] = df['location_name'].map(get_iso3_gbd)
df

,measure_name,location_name,metric_name,year,val,Country Code,country_code
12361,Incidence,Democratic People's Republic of Korea,Rate,1990,505.746115,PRK,PRK
12363,Incidence,Democratic People's Republic of Korea,Rate,1991,516.447635,PRK,PRK
12365,Incidence,Democratic People's Republic of Korea,Rate,1992,526.806601,PRK,PRK
12367,Incidence,Democratic People's Republic of Korea,Rate,1993,536.496942,PRK,PRK
12369,Incidence,Democratic People's Republic of Korea,Rate,1994,546.016220,PRK,PRK
...,...,...,...,...,...,...,...
24711,Incidence,Sierra Leone,Rate,2015,355.473682,SLE,SLE
24713,Incidence,Sierra Leone,Rate,2016,355.201957,SLE,SLE
24715,Incidence,Sierra Leone,Rate,2017,355.453785,SLE,SLE
24717,Incidence,Sierra Leone,Rate,2018,357.173519,SLE,SLE


In [7]:
df = df[['location_name', 'country_code', 'year', 'val']]
df.columns = ['Country Name', 'Country Code', 'Year', 'IHD']

In [8]:
print('======= GDB data ========')

======= GDB data ========


In [9]:
print('Number of countries:', df['Country Code'].nunique())

Number of countries: 206


In [10]:
print('GBD data year range:', df.Year.min(), df.Year.max())

GBD data year range: 1990 2019


In [11]:
print("Percentage missing data: ", df['IHD'].isna().mean())

Percentage missing data:  0.0


## Healthcare expenditure

In [12]:
year_cols = [str(i) for i in range(1960, 2023)]

In [13]:
health_df = pd.read_csv('../data/raw/oecd_healthSpending.csv')
health_df.rename(columns={'LOCATION': 'Country Code', 'TIME': 'Year', 'Value': 'HealthInd'}, inplace=True)
health_df.drop(columns=['INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'Flag Codes'], inplace=True)

In [14]:
print('======= Healthcare expenditure data ========')

======= Healthcare expenditure data ========


In [15]:
print('Healthcare expenditure data year range:', health_df.Year.min(), health_df.Year.max())

Healthcare expenditure data year range: 1970 2022


In [16]:
print('Number of countries:', health_df['Country Code'].nunique())

Number of countries: 50


In [17]:
print('% missing data:', health_df.HealthInd.isnull().mean())

% missing data: 0.0


In [18]:
print('% missing data from 1990 - 2019:', health_df[health_df.Year.isin(official_range)].HealthInd.isnull().mean())

% missing data from 1990 - 2019: 0.0


### GBD

In [18]:
global_burden_diseases = pd.read_csv('../data/raw/gbd_ischemicheartdiseaseglobal.csv')

In [19]:
print('======= IHD data ========')

======= IHD data ========


In [20]:
gbd = global_burden_diseases[global_burden_diseases.measure_name == 'Deaths']

In [21]:
gbd.groupby('location_name').year.nunique().min()

30

In [22]:
print('IHD data year range:', gbd.year.min(), gbd.year.max())

IHD data year range: 1990 2019


In [23]:
print('Number of countries:', gbd['location_name'].nunique())

Number of countries: 206


In [24]:
gbd.head()

,measure_id,measure_name,location_id,location_name,sex_id,sex_name,age_id,age_name,cause_id,cause_name,metric_id,metric_name,year,val,upper,lower
0,1,Deaths,82,Greece,3,Both,8,15-19 years,493,Ischemic heart disease,3,Rate,1990,0.299904,0.362727,0.245077
1,1,Deaths,82,Greece,3,Both,9,20-24 years,493,Ischemic heart disease,3,Rate,1990,2.819735,3.268572,2.390882
2,1,Deaths,82,Greece,3,Both,10,25-29 years,493,Ischemic heart disease,3,Rate,1990,4.226114,4.822198,3.692606
3,1,Deaths,82,Greece,3,Both,11,30-34 years,493,Ischemic heart disease,3,Rate,1990,7.773120,8.734354,6.820362
4,1,Deaths,82,Greece,3,Both,12,35-39 years,493,Ischemic heart disease,3,Rate,1990,15.752949,17.453460,14.201794


In [25]:
print('% missing data:', gbd.val.isnull().mean())

% missing data: 0.0


In [26]:
# global_burden_diseases.isnull().mean()

## Fat

In [27]:
def count_countries(series):
    print('Number of countries:', series.nunique())

def year_range(series):
    print('year range:', series.min(), series.max())

def count_missing(series):
    print('% missing data:', series.isnull().mean())

In [28]:
fat_df = pd.read_csv(DATA_PATH+'/daily_per_capita_fat_supply_final.csv')
fat_df = fat_df.rename({'Value': 'Fat'}, axis=1).drop(['Country Name', 'Series Name'], axis=1)

In [29]:
print('======= Fat consumption data ========')

======= Fat consumption data ========


In [30]:
count_countries(fat_df['Country Code'])
year_range(fat_df['Year'])
count_missing(fat_df[fat_df.Year.isin(official_range)]['Fat'])

Number of countries: 194
year range: 1961 2020
% missing data: 0.0


In [31]:
fat_df

,Country Code,Year,Fat
0,AFG,1961,38.209236
1,AFG,1962,38.335213
2,AFG,1963,39.338820
3,AFG,1964,39.747234
4,AFG,1965,40.560085
...,...,...,...
12355,ZWE,2016,66.536940
12356,ZWE,2017,62.848648
12357,ZWE,2018,64.250500
12358,ZWE,2019,56.008080


## Alcohol

In [32]:
alc_df = pd.read_csv('../data/raw/alcohol_germany.csv')
alc_df = alc_df.rename({'Value': 'Alcohol'}, axis=1)
alc_df

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Alcohol,Flag Codes
0,AUS,ALCOHOL,TOT,LT_CAP15,A,1960,9.3,NaN
1,AUS,ALCOHOL,TOT,LT_CAP15,A,1961,9.4,NaN
2,AUS,ALCOHOL,TOT,LT_CAP15,A,1962,9.5,NaN
3,AUS,ALCOHOL,TOT,LT_CAP15,A,1963,9.8,NaN
4,AUS,ALCOHOL,TOT,LT_CAP15,A,1964,10.1,NaN
...,...,...,...,...,...,...,...,...
2747,ROU,ALCOHOL,TOT,LT_CAP15,A,2015,9.5,NaN
2748,ROU,ALCOHOL,TOT,LT_CAP15,A,2016,9.6,NaN
2749,ROU,ALCOHOL,TOT,LT_CAP15,A,2017,10.1,NaN
2750,ROU,ALCOHOL,TOT,LT_CAP15,A,2018,11.0,NaN


In [33]:
print('======= Alcohol consumption data ========')

======= Alcohol consumption data ========


In [34]:
count_countries(alc_df['LOCATION'])
year_range(alc_df['TIME'])
count_missing(alc_df[alc_df.TIME.isin(official_range)]['Alcohol'])
alc_df[alc_df.TIME.isin(official_range)]['TIME'].unique()

Number of countries: 49
year range: 1960 2022
% missing data: 0.0


array([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019], dtype=int64)

## Age

In [35]:
age_df = pd.read_csv('../data/raw/median-age.csv')
age_df = age_df[['Code', 'Year', 'Median age - Sex: all - Age: all - Variant: estimates']]
age_df.columns = ['Country Code', 'Year', 'Age']

In [36]:
print('======= Age data ========')

======= Age data ========


In [37]:
count_countries(age_df['Country Code'])
year_range(age_df['Year'])
count_missing(age_df[age_df.Year.isin(official_range)]['Age'])

Number of countries: 238
year range: 1950 2100
% missing data: 0.0


In [38]:
# final_df = df.merge(health_df, on=['Country Code', 'Year'])
# final_df = final_df.merge(alc_df, on=['Country Code', 'Year'])
# final_df = final_df.merge(fat_df, on=['Country Code', 'Year'])
# final_df = final_df.merge(age_df, on=['Country Code', 'Year'])

In [39]:
# count_countries(final_df['Country Code'])